In [1]:
'''
Is this real time?  What are we doing with our random # of predictions?
So each day in the regular season is scored on one day.
- start at beginning of season, start with scoring day one of the season on today, when the script starts up

one week per day 
all pitches for the week (not rand 1-5)
start at the beginning of the data - 2016

There was change in data provider / collection means between the 2016 and 2017, an upgrade, 
but we're looking to see the model data drift
- so about 

Next step is to get it in real time.  We'll need to clean up the beautiful soup.
- eg, every morning at 8am, let's grab yesterday's data and score it
'''

"\nIs this real time?  What are we doing with our random # of predictions?\nSo each day in the regular season is scored on one day.\n- start at beginning of season, start with scoring day one of the season on today, when the script starts up\n\none week per day \nall pitches for the week (not rand 1-5)\nstart at the beginning of the data - 2016\n\nThere was change in data provider / collection means between the 2016 and 2017, an upgrade, \nbut we're looking to see the model data drift\n- so about \n\nNext step is to get it in real time.  We'll need to clean up the beautiful soup.\n- eg, every morning at 8am, let's grab yesterday's data and score it\n"

In [3]:
# Target is strike

import pandas as pd
import time
pd.options.display.max_columns = 500
t1 = time.time()
print('read')
df = pd.read_csv('pitch_scoring.csv', parse_dates=['date'])
print('- Done: %.3f' % (time.time()-t1))
print('shape:', df.shape)
df.head()

read
- Done: 5.677
shape: (705677, 33)


,date,ax,ay,az,batter_side,break_angle,break_length,break_y,end_speed,nasty,pfx_x,pfx_z,pitch_type,pitch_type_seq,pitcher_hand,px,pz,spin_dir,spin_rate,start_speed,strike,sz_bot,sz_top,type_conf,vx0,vy0,vz0,weather,wind,x0,y0,z0,zone
0,2018-03-29,-19.911162,32.791492,-17.557766,R,43.7,5.8,23.7,84.5,46.0,-10.721801,7.870604,SI,|SI,R,0.239615,1.848781,233.718,2632.270,93.7,True,1.564115,3.311768,2.0,6.687156,-136.134526,-7.015046,"73 degrees, partly cloudy","2 mph, Out to RF",-0.869064,50.0,5.696240,9.0
1,2018-03-29,-20.043108,30.911711,-18.780096,R,42.7,6.1,23.8,85.8,46.0,-10.647298,7.115135,SI,|SI|SI,R,-0.804047,0.925201,236.246,2561.181,94.0,False,1.601877,3.647501,2.0,4.093632,-136.551337,-9.145946,"73 degrees, partly cloudy","2 mph, Out to RF",-0.942066,50.0,5.617264,13.0
2,2018-03-29,-21.510485,30.321994,-18.731107,R,44.4,6.0,23.8,86.5,53.0,-11.206094,7.003230,SI,|SI|SI|SI,R,0.598637,1.616677,237.996,2673.854,94.7,True,1.426851,3.297457,2.0,7.501741,-137.628790,-7.507958,"73 degrees, partly cloudy","2 mph, Out to RF",-0.705522,50.0,5.646685,9.0
3,2018-03-29,-19.830780,31.121657,-20.898379,R,39.8,6.1,23.8,86.4,50.0,-10.283548,5.847167,SI,|SI,R,-0.161745,1.775260,240.377,2396.682,95.0,True,1.832977,3.866140,2.0,5.394874,-138.089373,-6.748109,"73 degrees, partly cloudy","2 mph, Out to RF",-0.806164,50.0,5.659253,13.0
4,2018-03-29,-7.686676,30.737312,-14.574052,R,26.5,3.6,23.8,83.7,42.0,-4.181113,9.573393,FF,|SI|FF,R,-0.829893,4.915535,203.592,2062.883,92.7,False,1.802699,3.929045,2.0,2.095397,-135.066745,0.341738,"73 degrees, partly cloudy","2 mph, Out to RF",-1.074593,50.0,5.816046,11.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705677 entries, 0 to 705676
Data columns (total 33 columns):
date              705677 non-null datetime64[ns]
ax                702956 non-null float64
ay                702956 non-null float64
az                702956 non-null float64
batter_side       705677 non-null object
break_angle       702956 non-null float64
break_length      702956 non-null float64
break_y           702956 non-null float64
end_speed         702988 non-null float64
nasty             702956 non-null float64
pfx_x             702979 non-null float64
pfx_z             702979 non-null float64
pitch_type        702956 non-null object
pitch_type_seq    705677 non-null object
pitcher_hand      705677 non-null object
px                702956 non-null float64
pz                702956 non-null float64
spin_dir          702956 non-null float64
spin_rate         702956 non-null float64
start_speed       702988 non-null float64
strike            705677 non-null bool
sz_bot 

In [5]:
df.strike.value_counts()

False    481362
True     224315
Name: strike, dtype: int64

In [6]:
sorted(df.columns.tolist())

['ax',
 'ay',
 'az',
 'batter_side',
 'break_angle',
 'break_length',
 'break_y',
 'date',
 'end_speed',
 'nasty',
 'pfx_x',
 'pfx_z',
 'pitch_type',
 'pitch_type_seq',
 'pitcher_hand',
 'px',
 'pz',
 'spin_dir',
 'spin_rate',
 'start_speed',
 'strike',
 'sz_bot',
 'sz_top',
 'type_conf',
 'vx0',
 'vy0',
 'vz0',
 'weather',
 'wind',
 'x0',
 'y0',
 'z0',
 'zone']

### Tristan script pred_pitches

In [7]:
import datetime
import os
import requests
import time

from dateutil import parser
import random
import numpy as np
import pandas as pd

# can grab all this from the Integrations tab
# TODO move to config file or add cli
API_TOKEN =  '-tthMRg4-sXF21D7FTQYxfUAMZSGWqvs' 
USERNAME = 'aengel@datarobot.com' 
DATAROBOT_KEY = '544ec55f-61bf-f6ee-0caf-15c7f919a45d'  
headers = {'Content-Type': 'application/json', 'datarobot-key': DATAROBOT_KEY}


CHAMPION_DEPLOYMENT_ID = '5bad2050e56fab405ccf6017'
# NOTE add a challenger deployment ID to route ~20% of requests to
CHALLENGER_DEPLOYMENT_ID = None

target = 'strike'
dt = 'date'

# prepare the data
t1 = time.time()
print('Reading csv')
fname = 'pitch_scoring.csv'
df = pd.read_csv(fname, parse_dates=[dt])
df = df.drop(target, axis=1)
print('- Done: %.3f' % (time.time()-t1))

df = df.sort_values(dt)
s_date = df[dt].min()
e_date = df[dt].max()

num_days = (e_date - s_date).days
cycles = num_days  # 14
days_per_cycle = num_days / cycles

periods = {}
print('Getting pitch data for %s cycles, %s days per cycle' % (cycles, days_per_cycle))
t1 = time.time()
for c in range(cycles):
    p_start = s_date + datetime.timedelta(days=days_per_cycle * c)
    p_end = p_start + datetime.timedelta(days=days_per_cycle)
    periods[c] = df[(df[dt] >= p_start) & (df[dt] <= p_end)]
print('- Done: %.3f' % (time.time()-t1))

Reading csv
- Done: 5.755
Getting pitch data for 516 cycles, 1.0 days per cycle
- Done: 1.911


In [9]:
print(s_date)
e_date

2017-04-02 00:00:00


Timestamp('2018-08-31 00:00:00')

In [70]:
MAX_ROWS = 5
DELAY_MIN = 0 # 8
DELAY_MAX = 2 # 12
# we want ~20k prediction rows a day which is ~14 per minute
# current avg will be be ~18 rows per minute

# NOTE change cycle_start to be an explict date if you stop + start the script but want to maintain the cycles.
# using utcnow() will start the cycle over again 
# start = datetime.datetime.utcnow()
start = datetime.datetime(2018, 9, 30)
end = start + datetime.timedelta(days=cycles)
now = datetime.datetime.utcnow()

print('start:',start)
print('now:  ',now)
print('end:  ',end)
print()

msg = "Starting prediction loop with settings...\n"
msg += "cycles: {} | days_per_cycle: {} | max_rows_per_req: {} | min_delay {} | max_delay {}".format(
    cycles, days_per_cycle, MAX_ROWS, DELAY_MIN, DELAY_MAX)
print(msg)

num_req = 0
total_rows = 0
while now < end:
    curr_cycle = (now - start).days
    num_rows = random.randint(1, MAX_ROWS)  # ---> Why sample a random number of rows from each cycle?
    total_rows += num_rows
    
    data = periods[curr_cycle]
    data = data.drop(dt, axis=1)

    # randomly sample some rows within this time periods
    cycle_sample_rows = data.sample(num_rows, random_state=0) 
    pred_data = cycle_sample_rows.to_json(orient='records')
    if random.random() > 0.2 or CHALLENGER_DEPLOYMENT_ID is None:
        predictions_response = requests.post('https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/%s/predictions' % (CHAMPION_DEPLOYMENT_ID),
                                        auth=(USERNAME, API_TOKEN), data=pred_data, headers=headers)

        predictions_response.raise_for_status()

    else:
        predictions_response = requests.post('https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/%s/predictions' % (CHALLENGER_DEPLOYMENT_ID),
                                         auth=(USERNAME, API_TOKEN), data=pred_data, headers=headers)
        predictions_response.raise_for_status()  # ---> what are we doing with predictions_response
    num_req += 1
    if num_req % 5 == 0 or num_req == 1:
        print("Current cycle: {} | Total Reqs: {} | Total Rows: {}".format(curr_cycle, num_req,
            total_rows))

    sleep = random.uniform(DELAY_MIN, DELAY_MAX)  # ---> why the random sleep
    time.sleep(sleep)
    now = datetime.datetime.utcnow()

num_days 516
cycles 14
days_per_cycle 36.857142857142854

Getting pitch data for 516 cycles
- Done: 1.981
start: 2018-09-30 00:00:00
now:   2018-10-04 05:54:02.850287
end:   2020-02-28 00:00:00

Starting prediction loop with settings...
cycles: 516 | days_per_cycle: 1 | max_rows_per_req: 5 | min_delay 0 | max_delay 2
Current cycle: 4 | Total Reqs: 1 | Total Rows: 1
Current cycle: 4 | Total Reqs: 5 | Total Rows: 12
Current cycle: 4 | Total Reqs: 10 | Total Rows: 29
Current cycle: 4 | Total Reqs: 15 | Total Rows: 45


KeyboardInterrupt: 

In [61]:
data.shape
random.randint(1, MAX_ROWS)
data.index.values

cycle_sample_rows = data.sample(num_rows, random_state=0) 
cycle_sample_rows
pred_data = cycle_sample_rows.to_json(orient='records')
pred_data

'[{"ax":-10.0996097363,"ay":28.9691335693,"az":-14.7263513787,"batter_side":"R","break_angle":29.7,"break_length":3.9,"break_y":23.8,"end_speed":85.8,"nasty":22.0,"pfx_x":-5.3622083159,"pfx_z":9.2635447648,"pitch_type":"FF","pitch_type_seq":"|SL|FF|FF|FF","pitcher_hand":"R","px":-1.946973232,"pz":2.1864536706,"spin_dir":210.063,"spin_rate":2148.068,"start_speed":93.6,"sz_bot":1.44,"sz_top":3.24,"type_conf":2.0,"vx0":5.7695767336,"vy0":-136.1579650446,"vz0":-4.1913952927,"weather":"48 degrees, overcast","wind":"19 mph, L to R","x0":-3.393431181,"y0":50.0,"z0":4.7597421164,"zone":13.0}]'

sleep for 11.677951758472721
done
Current cycle: 4 | Total Reqs: 30 | Total Rows: 109
sleep for 9.459994270902882
done
sleep for 9.371756485730877
done
sleep for 10.78383129251692
done
sleep for 8.883700585723059
done
sleep for 9.737358083745606
done
Current cycle: 4 | Total Reqs: 35 | Total Rows: 118
sleep for 8.238600477201505
done
sleep for 9.504848210413277
done
sleep for 9.320783611855706
done
sleep for 9.11522255661592
done
sleep for 10.391486160929464


KeyboardInterrupt: 